In [ ]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Which subreddit to generate wordclouds for
subreddit_name = 'The_Donald'

In [ ]:
def black_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    # Dummy function; always return black RGB value
    return "rgb(0, 0, 0)"

# Instantiate wordcloud generator
disc_mask = np.array(Image.open("blackdisc.jpeg"))
wc = WordCloud(font_path='OCR-A-Std-Regular.ttf',
               prefer_horizontal=1.0,
               background_color="white",
               mask=disc_mask,
               color_func=black_color_func)

In [ ]:
# Remember, V = WH
# W = document - latent
# H = latent - term

H = np.load('../clustering/results/H_{}.npy'.format(subreddit_name))
W = np.load('../clustering/results/W_{}.npy'.format(subreddit_name))
feature_names = np.load('../clustering/results/feature_names_{}.npy'.format(subreddit_name))

In [ ]:
classifs = W.argmax(axis=1)
tot_comments = W.shape[0]

mat = np.zeros([H.shape[0]])
for i in range(H.shape[0]):
    mat[i] = sum(classifs == i) / tot_comments

df = pd.DataFrame(data=np.atleast_2d(mat),
                  index=[subreddit_name],
                  columns=['Cluster {}'.format(i) for i in range(H.shape[0])])

In [ ]:
for topic_idx, topic in enumerate(H):
    print("Cluster {}:".format(topic_idx))
    
    d = {}
    for word_idx in topic.argsort()[:-15-1:-1]:
        d[feature_names[word_idx]] = topic[word_idx]
    
    wc.generate_from_frequencies(d)
    plt.subplots(figsize=[12, 12])
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.savefig('images/{}/{}_{:0.2f}%.png'.format(subreddit_name,
                                                   str(topic_idx).zfill(2),
                                                   100*df.loc[subreddit_name, 'Cluster {}'.format(topic_idx)]))
    plt.show()